In [ ]:
# IMPORTS

import json
import pandas as pd

In [ ]:
# Populate a list of our file names

filenum = 0
files = []

while filenum <21:
    filenum += 1
    file = f'raw_response_({filenum}).json'
    files.append(file)

In [ ]:
# E.T.L

for file in files: 
    with open(f'./data/{file}', encoding='utf-8') as df:
        alpha = json.load(df) # Pull the json data into a dictionary
        print(f'{file} has been loaded into the a dictionary...' ) # make sure we're looping
        cmnt_data = pd.DataFrame(alpha['comments']) # drill down to the coments in nested table 
        delta = pd.DataFrame(cmnt_data[['user_id','created_at_utc','text', 'comment_like_count', 'child_comment_count']])# comment data from 'replies' dict wth-in the df
        usr_data = pd.json_normalize(cmnt_data['user']) # collect the user data from the 'comment' dictionary within the df
        usr_tbl =  usr_data[['pk', 'full_name', 'is_private', 'username', 'is_verified']] # pull the column data we want about the users
        cmt_tbl = delta[['user_id', 'created_at_utc', 'text', 'comment_like_count', 'child_comment_count']] # pull the column data we want about the comments
        staging_tabble = pd.concat([usr_tbl, cmt_tbl], axis=1) # hold the transformed data for writing to the final table
    insta_comments_table = pd.concat([staging_tabble, insta_comments_table], axis=0, ignore_index=True) # save the staging_table to the final df

insta_comments_table

In [ ]:
# Save

insta_comments_table.to_csv('instagram_data.csv')